## Comments

<hr>

# T2I Train scripts

Install necessary libraries

In [ ]:
!pip install torch==2.0.1 --force-reinstall

In [ ]:
!pip install pytorch_lightning==1.06

In [ ]:
!pip install opencv-python basicsr omegaconf datasets diffusers transformers einops xformers==0.0.22 open-clip-torch==2.24.0

In [ ]:
!pip install torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/cu117

Environment tests

In [ ]:
!nvidia-smi

Test if modifying amount of memory for Torch with 'set_per_process_memory_fraction(0.5, 0)' works

In [ ]:
# https://github.com/pytorch/pytorch/blob/47aa2536328afc51876b2e04384c0cfe71ee1f06/test/test_cuda.py#L394-L421
import torch 

tensor = torch.zeros(1024, device='cuda')
torch.cuda.empty_cache()
total_memory = torch.cuda.get_device_properties(0).total_memory
torch.cuda.set_per_process_memory_fraction(0.5, 0)

# test 0.499 allocation is ok.
application = int(total_memory * 0.499) - torch.cuda.max_memory_reserved()
tmp_tensor = torch.empty(application, dtype=torch.int8, device='cuda')
del tmp_tensor
torch.cuda.empty_cache()

application = int(total_memory * 0.5)
# it will get OOM when try to allocate more than half memory.
with self.assertRaisesRegex(RuntimeError, "out of memory"):
    torch.empty(application, dtype=torch.int8, device='cuda')

# ensure out of memory error doesn't disturb subsequent kernel
tensor.fill_(1)
self.assertTrue((tensor == 1).all())

In [ ]:
import torch
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.device(0), torch.cuda.get_device_name(0)

In [ ]:
device = 1
t = torch.cuda.get_device_properties(device).total_memory
r = torch.cuda.memory_reserved(device)
a = torch.cuda.memory_allocated(device)
f = r-a  # free inside reserved
t, r, a, f

In [ ]:
import gc

gc.collect()

import torch
torch.cuda.empty_cache() 
del gc

## Create Dataset (LLAVA + SHIFT)

In [ ]:
!python T2I-Adapter/sample_images_train.py

In [ ]:
!python T2I-Adapter/llava_dataset_creation.py

## 1 Condition

### train_sketch.py

In [ ]:
#!accelerate launch T2I-Adapter/train_sketch.py 
#!python -m torch.distributed.launch T2I-Adapter/train_sketch.py --bsize 1 --epochs 1 --num_workers 1 --n_samples 1 --scale 1 --gpus 1
# H = Height pixels / 4
# W = Width pixels / 4
!torchrun T2I-Adapter/train_sketch.py --local_rank 1 --bsize 1 --epochs 5 --num_workers 1 --n_samples 1 --scale 1 --gpus 1 --H 256 --W 256 

* Samples are created in  T2I-Adapter/experiments/train-sketch/visualization

#### Running it in notebook

In [ ]:
import argparse
import logging
import os
import os.path as osp
import time
os.chdir("T2I-Adapter")
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from basicsr.utils import (
    get_env_info,
    get_root_logger,
    get_time_str,
    img2tensor,
    scandir,
    tensor2img,
)
from basicsr.utils.options import copy_opt_file, dict2str
from omegaconf import OmegaConf
from PIL import Image

# from ldm.data.dataset_coco import dataset_coco_mask_color
from ldm.data.dataset_fill import dataset_fill_mask

from dist_util import get_bare_model, get_dist_info, init_dist, master_only
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.dpm_solver import DPMSolverSampler
from ldm.models.diffusion.plms import PLMSSampler
from ldm.modules.encoders.adapter import Adapter
from ldm.util import instantiate_from_config
from ldm.modules.extra_condition.model_edge import pidinet



def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model


@master_only
def mkdir_and_rename(path):
    """mkdirs. If path exists, rename it with timestamp and create a new one.

    Args:
        path (str): Folder path.
    """
    if osp.exists(path):
        new_name = path + "_archived_" + get_time_str()
        print(f"Path already exists. Rename it to {new_name}", flush=True)
        os.rename(path, new_name)
    os.makedirs(path, exist_ok=True)
    os.makedirs(osp.join(experiments_root, "models"))
    os.makedirs(osp.join(experiments_root, "training_states"))
    os.makedirs(osp.join(experiments_root, "visualization"))


def load_resume_state(opt):
    resume_state_path = None
    if opt.auto_resume:
        state_path = osp.join("experiments", opt.name, "training_states")
        if osp.isdir(state_path):
            states = list(
                scandir(state_path, suffix="state", recursive=False, full_path=False)
            )
            if len(states) != 0:
                states = [float(v.split(".state")[0]) for v in states]
                resume_state_path = osp.join(state_path, f"{max(states):.0f}.state")
                opt.resume_state_path = resume_state_path
    # else:
    #     if opt['path'].get('resume_state'):
    #         resume_state_path = opt['path']['resume_state']
    if resume_state_path is None:
        resume_state = None
    else:
        device_id = torch.cuda.current_device()
        resume_state = torch.load(
            resume_state_path, map_location=lambda storage, loc: storage.cuda(device_id)
        )
        # check_resume(opt, resume_state['iter'])
    return resume_state


In [ ]:
class Opt:
    def __init__(self):
        # Default attribute values
        self.bsize = 8
        self.epochs = 10000
        self.num_workers = 8
        self.use_shuffle = True
        self.dpm_solver = False
        self.plms = False
        self.auto_resume = False
        self.ckpt = "T2I-Adapter/models/sd-v1-4.ckpt"
        self.config = "configs/stable-diffusion/train_sketch.yaml"
        self.print_fq = 100  # TODO:
        self.H = 512
        self.W = 512
        self.C = 4
        self.f = 8
        self.ddim_steps = 50
        self.n_samples = 1
        self.ddim_eta = 0.0
        self.scale = 7.5
        self.gpus = [0, 1, 2, 3]
        self.local_rank = 0
        self.launcher = "pytorch"
        self.l_cond = 4


In [ ]:
opt = Opt()

print("-> Configuration from", opt.config)
config = OmegaConf.load(f"{opt.config}")
opt.name = config["name"]


In [ ]:
# distributed setting
print("-> Distributed setting")
init_dist(opt.launcher)
torch.backends.cudnn.benchmark = True
device = "cuda"
torch.cuda.set_device(opt.local_rank)

In [ ]:
# dataset
print("-> Dataset")
# FIXME: previous dataset coco2017
# path_json_train = "coco_stuff/mask/annotations/captions_train2017.json"
# path_json_val = "coco_stuff/mask/annotations/captions_val2017.json"
# train_dataset = dataset_coco_mask_color(
#     path_json_train,
#     root_path_im="coco/train2017",
#     root_path_mask="coco_stuff/mask/train2017_color",
#     image_size=512,
# )
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
# val_dataset = dataset_coco_mask_color(
#     path_json_val,
#     root_path_im="coco/val2017",
#     root_path_mask="coco_stuff/mask/val2017_color",
#     image_size=512,
# )
# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=opt.bsize,
#     shuffle=(train_sampler is None),
#     num_workers=opt.num_workers,
#     pin_memory=True,
#     sampler=train_sampler,
# )
# val_dataloader = torch.utils.data.DataLoader(
#     val_dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=False
# )

# Add data and validation loader with very small dataset
print("-> Add data and validation loader with very small dataset")
train_dataset = dataset_fill_mask(split="train")
train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
val_dataset = dataset_fill_mask(split="train")
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=opt.bsize,
    shuffle=(train_sampler is None),
    num_workers=opt.num_workers,
    pin_memory=True,
    sampler=train_sampler,
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=False
)

In [ ]:
# edge_generator
os.chdir("..")
print("-> Edge generator")
net_G = pidinet()
ckp = torch.load("T2I-Adapter/models/table5_pidinet.pth", map_location="cpu")[
    "state_dict"
]
net_G.load_state_dict({k.replace("module.", ""): v for k, v in ckp.items()})
net_G.cuda()

In [ ]:
# stable diffusion
print("-> Stable diffusion")
model = load_model_from_config(config, f"{opt.ckpt}").to(device)

# sketch encoder
print("-> Sketch encoder")
model_ad = Adapter(
    channels=[320, 640, 1280, 1280][:4], nums_rb=2, ksize=1, sk=True, use_conv=False
).to(device)

# # to gpus
print("-> to gpus")
model_ad = torch.nn.parallel.DistributedDataParallel(
    model_ad, device_ids=[opt.local_rank], output_device=opt.local_rank
)
model = torch.nn.parallel.DistributedDataParallel(
    model, device_ids=[opt.local_rank], output_device=opt.local_rank
)
# device_ids=[torch.cuda.current_device()])
net_G = torch.nn.parallel.DistributedDataParallel(
    net_G, device_ids=[opt.local_rank], output_device=opt.local_rank
)
# device_ids=[torch.cuda.current_device()])

# optimizer
print("-> optimizer")
params = list(model_ad.parameters())
optimizer = torch.optim.AdamW(params, lr=config["training"]["lr"])

experiments_root = osp.join("T2I-Adapter/experiments", opt.name)

In [ ]:
# resume state
print("-> resume state")
resume_state = load_resume_state(opt)
if resume_state is None:
    mkdir_and_rename(experiments_root)
    start_epoch = 0
    current_iter = 0
    # WARNING: should not use get_root_logger in the above codes, including the called functions
    # Otherwise the logger will not be properly initialized
    log_file = osp.join(experiments_root, f"train_{opt.name}_{get_time_str()}.log")
    logger = get_root_logger(
        logger_name="basicsr", log_level=logging.INFO, log_file=log_file
    )
    logger.info(get_env_info())
    logger.info(dict2str(config))
else:
    # WARNING: should not use get_root_logger in the above codes, including the called functions
    # Otherwise the logger will not be properly initialized
    log_file = osp.join(experiments_root, f"train_{opt.name}_{get_time_str()}.log")
    logger = get_root_logger(
        logger_name="basicsr", log_level=logging.INFO, log_file=log_file
    )
    logger.info(get_env_info())
    logger.info(dict2str(config))
    resume_optimizers = resume_state["optimizers"]
    optimizer.load_state_dict(resume_optimizers)
    logger.info(
        f"Resuming training from epoch: {resume_state['epoch']}, "
        f"iter: {resume_state['iter']}."
    )
    start_epoch = resume_state["epoch"]
    current_iter = resume_state["iter"]

In [ ]:

# copy the yml file to the experiment root
print("-> copy the yml file to the experiment root")
os.chdir("T2I-Adapter")
# copy_opt_file(opt.config, experiments_root) # TODO: for now we copy yaml file by hand to experiments/


In [ ]:
# training
print("-> Training & validation")
logger.info(f"Start training from epoch: {start_epoch}, iter: {current_iter}")
for epoch in range(start_epoch, opt.epochs):
    logger.info(f"Epoch: {epoch}")
    train_dataloader.sampler.set_epoch(epoch)
    # train
    for _, data in enumerate(train_dataloader):
        current_iter += 1
        with torch.no_grad():
            edge = net_G(data["im"].cuda(non_blocking=True))[-1]
            edge = edge > 0.5
            edge = edge.float()
            c = model.module.get_learned_conditioning(data["sentence"]) 
            z = model.module.encode_first_stage( 
                (data["im"] * 2 - 1.0).cuda(non_blocking=True)
            )
            z = model.module.get_first_stage_encoding(z) 

        optimizer.zero_grad()
        model.zero_grad()
        features_adapter = model_ad(edge)
        l_pixel, loss_dict = model(z, c=c, features_adapter=features_adapter)
        # l_pixel.backward() #FIXME: takes too much space
        optimizer.step()

        if (current_iter + 1) % opt.print_fq == 0:
            logger.info(loss_dict)

        # save checkpoint
        rank, _ = get_dist_info()
        if (rank == 0) and (
            (current_iter + 1) % float(config["training"]["save_freq"]) == 0
        ):
            save_filename = f"model_ad_{current_iter+1}.pth"
            save_path = os.path.join(experiments_root, "models", save_filename)
            save_dict = {}
            model_ad_bare = get_bare_model(model_ad)
            state_dict = model_ad_bare.state_dict()
            for key, param in state_dict.items():
                if key.startswith("module."):  # remove unnecessary 'module.'
                    key = key[7:]
                save_dict[key] = param.cpu()
            torch.save(save_dict, save_path)
            # save state
            state = {
                "epoch": epoch,
                "iter": current_iter + 1,
                "optimizers": optimizer.state_dict(),
            }
            save_filename = f"{current_iter+1}.state"
            save_path = os.path.join(
                experiments_root, "training_states", save_filename
            )
            torch.save(state, save_path)

    # val
    rank, _ = get_dist_info()
    if rank == 0:
        for data in val_dataloader:
            with torch.no_grad():
                if opt.dpm_solver:
                    sampler = DPMSolverSampler(model.module)
                elif opt.plms:
                    sampler = PLMSSampler(model.module)
                else:
                    sampler = DDIMSampler(model.module)
                print(data["im"].shape)
                c = model.module.get_learned_conditioning(data["sentence"])
                edge = net_G(data["im"].cuda(non_blocking=True))[-1]
                edge = edge > 0.5
                edge = edge.float()
                im_edge = tensor2img(edge)
                cv2.imwrite(
                    os.path.join(
                        experiments_root, "visualization", "edge_%04d.png" % epoch
                    ),
                    im_edge,
                )
                print(
                    "-> Edge saved in ",
                    os.path.join(
                        experiments_root, "visualization", "edge_%04d.png" % epoch
                    ),
                )
                features_adapter = model_ad(edge)
                shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                samples_ddim, _ = sampler.sample(
                    S=opt.ddim_steps,
                    conditioning=c,
                    batch_size=opt.n_samples,
                    shape=shape,
                    verbose=False,
                    unconditional_guidance_scale=opt.scale,
                    unconditional_conditioning=model.module.get_learned_conditioning(
                        opt.n_samples * [""]
                    ),
                    eta=opt.ddim_eta,
                    x_T=None,
                    features_adapter=features_adapter,
                )
                x_samples_ddim = model.module.decode_first_stage(samples_ddim)
                x_samples_ddim = torch.clamp(
                    (x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0
                )
                x_samples_ddim = x_samples_ddim.cpu().permute(0, 2, 3, 1).numpy()
                for id_sample, x_sample in enumerate(x_samples_ddim):
                    x_sample = 255.0 * x_sample
                    img = x_sample.astype(np.uint8)
                    img = cv2.putText(
                        img.copy(),
                        data["sentence"][0],
                        (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 255, 0),
                        2,
                    )
                    cv2.imwrite(
                        os.path.join(
                            experiments_root,
                            "visualization",
                            "sample_e%04d_s%04d.png" % (epoch, id_sample),
                        ),
                        img[:, :, ::-1],
                    )
                    print("-> Image generated in:", os.path.join(
                            experiments_root,
                            "visualization",
                            "sample_e%04d_s%04d.png" % (epoch, id_sample),
                        ))
            break


### train_seg.py

In [ ]:
!torchrun T2I-Adapter/train_seg.py --local_rank 1 --bsize 1 --epochs 1 --num_workers 1 --n_samples 1 --scale 1 --gpus 1 --H 256 --W 256

### train_depth.py

In [ ]:
!torchrun T2I-Adapter/train_depth.py --local_rank 1 --bsize 1 --epochs 1 --num_workers 1 --n_samples 1 --scale 1 --gpus 1 --H 256 --W 256

## 2 Conditions


In [ ]:
!python T2I-Adapter/train.py -t --base configs/pl_train/coadapter-v1-train.yaml --train True --no-test True --gpus 1 --gpu 1 --scale_lr False \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --adapter_finetune_from models/t2iadapter_canny_sd14v1.pth \
    --coadapter_finetune_from models/t2iadapter_depth_sd14v1.pth 
    #--name coadapter-v1-train --autoresume

## 3 Conditions

### Train

BE ADVISED: I did not use open pose, key pose or style conditionings since these do not seem apropiate for the road-image generation task (see [here](https://github.com/TencentARC/T2I-Adapter/blob/SD/docs/examples.md#style-adapter)). Canny is also discarded due to its similarity to sketch conditioning

WARNING: First I remove all checkpoints to train them from scratch 

In [103]:
!rm -rf T2I-Adapter/logs/*

Depth, Segmentation Mask, Sketch

In [104]:
!python T2I-Adapter/train.py -t --base configs/pl_train/coadapter-v1-train_depth_seg_sketch.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr True \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --adapter_finetune_from models/t2iadapter_seg_sd14v1.pth \
    --coadapter_finetune_from models/t2iadapter_depth_sd14v1.pth \
    --coadapter2_finetune_from models/t2iadapter_sketch_sd14v1.pth \
    --name coadapter-depth-seg-sketch 
    

Color, Segmentation Mask, Sketch

In [ ]:
!python T2I-Adapter/train.py -t --base configs/pl_train/coadapter-v1-train_color_seg_sketch.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr True \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --adapter_finetune_from models/t2iadapter_seg_sd14v1.pth \
    --coadapter_finetune_from models/t2iadapter_color_sd14v1.pth \
    --coadapter2_finetune_from models/t2iadapter_sketch_sd14v1.pth \
    --name coadapter-color-seg-sketch \
    

Color, Depth, Segmentation Mask 

In [ ]:
!python T2I-Adapter/train.py -t --base configs/pl_train/coadapter-v1-train_color_depth_seg.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr True \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --adapter_finetune_from models/t2iadapter_seg_sd14v1.pth \
    --coadapter_finetune_from models/t2iadapter_depth_sd14v1.pth \
    --coadapter2_finetune_from models/t2iadapter_color_sd14v1.pth \
    --name coadapter-color-depth-seg \
    

Color, Depth, Sketch

In [ ]:
!python T2I-Adapter/train.py -t --base configs/pl_train/coadapter-v1-train_color_depth_sketch.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr False \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --adapter_finetune_from models/t2iadapter_color_sd14v1.pth \
    --coadapter_finetune_from models/t2iadapter_depth_sd14v1.pth \
    --coadapter2_finetune_from models/t2iadapter_sketch_sd14v1.pth \
    --name coadapter-color-depth-sketch \
    

### Train 2

BE ADVISED: I did not use open pose, key pose or style conditionings since these do not seem apropiate for the road-image generation task (see [here](https://github.com/TencentARC/T2I-Adapter/blob/SD/docs/examples.md#style-adapter)). Canny is also discarded due to its similarity to sketch conditioning

WARNING: First I remove all checkpoints to train them from scratch 

In [146]:
!rm -rf T2I-Adapter/logs/*

Color, depth, sketch

In [ ]:
!python T2I-Adapter/train2.py -t --base configs/pl_train/coadapter-v1-train_color_depth_sketch.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr True \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --coadapter_finetune_from models/coadapter-fuser-sd15v1.pth \
    --name coadapter-color-depth-sketch-short-prompt

Depth, Segmentation Mask, Sketch

In [ ]:
!python T2I-Adapter/train2.py -t --base configs/pl_train/coadapter-v1-train_depth_seg_sketch.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr True \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --coadapter_finetune_from models/coadapter-fuser-sd15v1.pth \
    --name coadapter-depth-seg-sketch-long-prompt

Color, Segmentation Mask, Sketch

In [ ]:
!python T2I-Adapter/train2.py -t --base configs/pl_train/coadapter-v1-train_color_seg_sketch.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr True \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --coadapter_finetune_from models/coadapter-fuser-sd15v1.pth \
    --name coadapter-color-seg-sketch-short-prompt

Color, Depth, Segmentation Mask 

In [ ]:
!python T2I-Adapter/train2.py -t --base configs/pl_train/coadapter-v1-train_color_depth_seg.yaml --train True --test False --gpus 1 --gpu 0 --scale_lr True \
    --num_nodes 1 --sd_finetune_from models/sd-v1-4.ckpt \
    --coadapter_finetune_from models/coadapter-fuser-sd15v1.pth \
    --name coadapter-color-depth-seg-short-prompt

### Plot loss vs epoch

In [153]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_losses(csv_file):
    # Read the CSV file into a pandas DataFrame, ignoring rows with missing values
    df = pd.read_csv(csv_file).dropna(subset=['train/loss_simple_epoch', 'train/loss_vlb_epoch', 'train/loss_epoch', 'epoch'])

    # Extract relevant columns
    epochs = df['epoch']
    loss_simple = df['train/loss_simple_epoch']
    loss_vlb = df['train/loss_vlb_epoch']
    loss = df['train/loss_epoch']

    # Create figure and axes for subplots
    fig, axes = plt.subplots(2, 1, figsize=(10, 10), sharex=True)

    # Plot each type of loss in separate subplot
    axes[0].plot(epochs, loss_simple, label='Loss Simple', color='b')
    axes[0].set_ylabel('Loss Simple')
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(epochs, loss_vlb, label='Loss VLB', color='g')
    axes[1].set_ylabel('Loss VLB')
    axes[1].legend()
    axes[1].grid(True)
    """
    axes[2].plot(epochs, loss, label='Loss', color='r')
    axes[2].set_ylabel('Loss')
    axes[2].set_xlabel('Epoch')
    axes[2].legend()
    axes[2].grid(True)
    """
    plt.tight_layout()
    plt.show()


In [ ]:
plot_losses("T2I-Adapter/logs/coadapter-color-depth-sketch-long-prompt/csvlogger/version_0/metrics.csv")

Save to targz

In [ ]:
!tar chvfz T2I-Adapter/logs/logs.tar.gz T2I-Adapter/logs/*

### Generation

#### Script 1: uses SD (and adapters) v. 1.4 by default

In [ ]:
# From SD 1.4 vainilla, without training
#!python T2I-Adapter/app.py --sd_ckpt T2I-Adapter/models/sd-v1-4.ckpt

In [ ]:
# With gradio
#!python T2I-Adapter/app.py

In [ ]:
!python T2I-Adapter/app_script.py

#### Script 2: uses SD (and adapters) v. 1.5 by default but works with SD 1.4

In [ ]:
# From SD 1.4 vainilla, without training
#!python T2I-Adapter/app_coadapter.py --sd_ckpt T2I-Adapter/models/sd-v1-4.ckpt

In [ ]:
# From training done above. Checkpoint saved in last.ckpt
!python T2I-Adapter/app_coadapter.py --sd_ckpt T2I-Adapter/logs/coadapter-color-depth-sketch/checkpoints/last.ckpt

### See metrics and results

Stored in "test/" and "test/metrics.csv"

In [2]:
import pandas as pd
import ipyplot

### Long prompt

#### Conditionings results

In [3]:
metrics = pd.read_csv("T2I-Adapter/dataset/test_long_prompt/metrics.csv")
labels = pd.read_csv("T2I-Adapter/dataset/train.csv")

In [ ]:
metrics.loc[:,["Conditionings", "MSE","Perceptual_Loss"]].groupby("Conditionings").mean().sort_values(by=["MSE", "Perceptual_Loss"])

Best images

In [ ]:
metrics.loc[:,["Image", "MSE","Perceptual_Loss"]].groupby("Image").mean().sort_values(by=["MSE", "Perceptual_Loss"])

Best conditionings+weights for each metric

In [ ]:
metrics.iloc[metrics["MSE"].argmin(),:], metrics.iloc[metrics["Perceptual_Loss"].argmin(),:]

In [ ]:
metrics.iloc[metrics["MSE"].argmax(),:], metrics.iloc[metrics["Perceptual_Loss"].argmax(),:]

#### Best example for MSE and Perceptual Loss:

In [9]:
def plot_example(img, conds, weights, plot_conds=False):
    images_list = [f'T2I-Adapter/dataset/train/{img}',
                   f'T2I-Adapter/dataset/test/{conds}_{weights}/{img}_generated_0.png']
    conds_list = [f'T2I-Adapter/dataset/test/{conds}_{weights}/{img}_{conds.split("_")[0]}.png',
                 f'T2I-Adapter/dataset/test/{conds}_{weights}/{img}_{conds.split("_")[1]}.png',
                 f'T2I-Adapter/dataset/test/{conds}_{weights}/{img}_{conds.split("_")[2]}.png']

    ipyplot.plot_images(images_list, max_images=20, img_width=500)
    if plot_conds:
        ipyplot.plot_images(conds_list, max_images=20, img_width=350)
    print(labels[labels.Image == img].Labels.values)
    print(metrics[(metrics.Image==img) & (metrics.Conditionings == conds) & (metrics.Weights == weights)].loc[:,["MSE","Perceptual_Loss"]])

In [ ]:
plot_example("00000500_img_front.jpg", "color_seg_sketch", "1.0_0.5_1.0")

In [ ]:
plot_example("00000360_img_front.jpg", "color_seg_sketch", "0.5_0.5_1.0")

#### Best examples for best overall conditionings

In [ ]:
mse_min = metrics[metrics.Conditionings=="color_depth_sketch"]["MSE"].argmin()
pl_min = metrics[metrics.Conditionings=="color_depth_sketch"]["Perceptual_Loss"].argmin()

metrics[metrics.Conditionings=="color_depth_sketch"].iloc[mse_min,:], metrics[metrics.Conditionings=="color_depth_sketch"].iloc[pl_min,:]

In [ ]:
plot_example("00000500_img_front.jpg", "color_depth_sketch", "1.0_0.5_0.5")

In [ ]:
plot_example("00000360_img_front.jpg", "color_depth_sketch", "1.0_1.0_0.5")

#### Worse examples

In [ ]:
mse_max = metrics[metrics.Conditionings=="color_depth_sketch"]["MSE"].argmax()
pl_max = metrics[metrics.Conditionings=="color_depth_sketch"]["Perceptual_Loss"].argmax()

metrics[metrics.Conditionings=="color_depth_sketch"].iloc[mse_max,:], metrics[metrics.Conditionings=="color_depth_sketch"].iloc[pl_min,:]

In [ ]:
plot_example("00000500_img_front.jpg", "color_depth_sketch", "1.0_0.5_0.5")

In [ ]:
plot_example("00000500_img_front.jpg", "color_depth_sketch", "1.0_0.5_0.5")

#### Some examples:

In [ ]:
from numpy import random
conditionings = metrics.loc[:,"Conditionings"].unique()
weights = metrics.loc[:,"Weights"].unique()
images = metrics.loc[:,"Image"].unique()


NUM_IMAGES = 10

for cond, w, img in zip(random.choice(conditionings, NUM_IMAGES), random.choice(weights, NUM_IMAGES), random.choice(images, NUM_IMAGES)):
    plot_example(img, cond, w, True)